# chapter 5 - Recommendation Systems
추천 시스템
1. 협업 필터링 (CF) Collaborative Filtering 
2. 콘텐츠 기반 필터링 (CBF) Content-based Filtering

기타 접근방법
1. 연관 룰 association rules
2. $Log$ 우도 the log-likelihood method
3. 하이브리드 hybrid methods

<br></br>
## 1 유틸리티 행렬
Utility matrix 추천시스템에 사용되는 계량척도 시스템

평점 사용자 $i$가 아이템 $j$를 평가한 사용자 평가목록 : $ r_{ij}$ 

### 01 데이터 수집 및 전처리
import Csv files 

In [1]:
import numpy as np
import pandas as pd
from time import time
from scipy import linalg
from collections import defaultdict
import copy, collections, math

# 사용자 평점 데이터 불러오기
df = pd.read_csv('./data/ml-100k/u.data', sep='\t', header=None)

num_users  = len(df[0].drop_duplicates().tolist())  
movies_rated  = list(df[1]) 
counts = collections.Counter(movies_rated)
df.head(2)

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742


In [2]:
# https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
# 데이터 영화목록 불러오기
df_info = pd.read_csv('./data/ml-100k/u.item', sep='|', header=None, encoding = "ISO-8859-1")

# 영화목록 추출
movie_list = [df_info[1].tolist()[ indx ] + ';' + str(indx+1) 
              for indx in range(len(df_info[1].tolist())) ]
num_movies = len(movie_list)
df_info.head(2)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
df_out = pd.DataFrame(columns=['user']+movie_list)
df_out  # 영화의 목록을 컬럼으로 생성

Empty DataFrame
Columns: [user, Toy Story (1995);1, GoldenEye (1995);2, Four Rooms (1995);3, Get Shorty (1995);4, Copycat (1995);5, Shanghai Triad (Yao a yao yao dao waipo qiao) (1995);6, Twelve Monkeys (1995);7, Babe (1995);8, Dead Man Walking (1995);9, Richard III (1995);10, Seven (Se7en) (1995);11, Usual Suspects, The (1995);12, Mighty Aphrodite (1995);13, Postino, Il (1994);14, Mr. Holland's Opus (1995);15, French Twist (Gazon maudit) (1995);16, From Dusk Till Dawn (1996);17, White Balloon, The (1995);18, Antonia's Line (1995);19, Angels and Insects (1995);20, Muppet Treasure Island (1996);21, Braveheart (1995);22, Taxi Driver (1976);23, Rumble in the Bronx (1995);24, Birdcage, The (1996);25, Brothers McMullen, The (1995);26, Bad Boys (1995);27, Apollo 13 (1995);28, Batman Forever (1995);29, Belle de jour (1967);30, Crimson Tide (1995);31, Crumb (1994);32, Desperado (1995);33, Doom Generation, The (1995);34, Free Willy 2: The Adventure Home (1995);35, Mad Love (1995);36, Nadja (199

In [4]:
# 사용자 리뷰중 50개 이하의 영화 리뷰를 작성한 사람들의 데이터는 제거한다
min_ratings = 50  
to_removelist = []; t0 = time()

for i in range(1,num_users):
    tmp_movielist = [0 for j in range(num_movies)]
    df_tmp = df[df[0] == i]
    for k in df_tmp.index:
        if counts[ df_tmp.ix[k][1] ] >= min_ratings:
            tmp_movielist[ df_tmp.ix[k][1] -1 ] = df_tmp.ix[k][2]
        else: to_removelist.append(df_tmp.ix[k][1])
    df_out.loc[i] = [i] + tmp_movielist   # df의 사용자 리뷰 데이터를 df_out 에 첨부

to_remove_list = list(set(to_removelist))
df_out.drop(df_out.columns[ to_removelist ], axis = 1, inplace = True)
df_out.to_csv('data/utilitymatrix.csv', index = None)
print(round(time()-t0,4),'sec'); df_out.iloc[:4,:6]

32.6869 sec


In [ ]:
df_out.shape
# 총 1683개의 영화에 대한 개별 사용자의 평점을 첨부한다
# 사용자가 모든 영화에 대한 리뷰를 작성하지는 않으므로, 50개 이하의 평점이 기록된 영화는 제거한다
# NaN 결측치에 대하여, 정상적인 작동을 위해 '대체'(imputation) 의 방법으로 결측치 초기화를 한다

### 02 결측치 처리
대체 (imputation) : <strong>사용자의 평균 평점</strong>과 <strong>영화별 평균 평점</strong>을 대체에 사용한다

In [12]:
def imputation(inp,Ri):
    Ri = Ri.astype(float)
    def userav():
        for i in range(len(Ri)):
            Ri[i][Ri[i] == 0] = sum( Ri[i] ) / float(len(Ri[i][Ri[i] > 0]))
        return Ri
    def itemav():
        for i in range(len(Ri[0])):
            Ri[:, i][Ri[:, i] == 0] = sum(Ri[:,i])/float(len(Ri[:, i][Ri[:, i] > 0]))
        return Ri            
    switch = {'useraverage' : userav() ,'itemaverage' : itemav() }
    return switch[inp]

<br></br>
## 2 유사도 척도
벡터 $X$와 $Y$의 유사도를 계산하기 위한 척도
1. 코사인 유사도 Cosine similarity
2. 피어슨 상관관계 Pearson correlation
두 척도의 평균이 0인경우, 일치하게 된다

In [13]:
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine 
def similarity(x,y,metric='cos'):
    if metric == 'cos':
        return 1.-cosine(x,y)   # 코사인 유사도를 출력
    else:
        return pearsonr(x,y)[0] # 피어슨 상관계수를 출력

<br></br>
## 3 협업 필터링 방법
<h4><strong>Collaborative Filtering methods</strong></h4>
"자신과 비슷한 사람들은, 좋아하는 아이템을 공유한다"를 기반으로 한다

이는 사용자의 수많은 데이터를 필요로 하므로 <strong>Cold Start</strong>

(<strong>가용 데이터 부족</strong>상태로, 반대로 <strong>Warm Start</strong>가 있다)가 문제가 되고 있다

이를 극복하기 위하여, <strong>CF</strong>와 <strong>CBF의 하이브리드</strong> 방식이 제안되곤 한다

(콘텐츠의 내부적 분류로써, 사용자의 데이터를 더 세분화 한 데이터로 분석을 한다)
<h4><strong>콘텐츠 기반 알고리즘의 이슈</strong></h4>
공통이슈로 <strong>'확장성(Scalability)'</strong>이 언급되는데

이는 사용자와 아이템 갯수에 비례하여 연산량이 급증하는 문제를 말한다(<strong>병렬처리</strong>가 필요)

반면 아이템 갯수가 작은경우에는 <strong>희소성(Sparsity)</strong> 문제가 생긴다(<strong>대체</strong>의 방법으로 해결


<h4><strong>1 메모리 기반의 협업 필터링</strong></h4>
Memory-based Collaborative Filtering 은 <strong>유틸리티 행렬</strong>을 사용하여 아이템의 유사도를 계산한다
<h4><strong>2 사용자 기반 협업 필터링</strong></h4>
User-based Collaborative Filtering 은 <strong> K-NN 알고리즘</strong>을 사용한다, 

유사 사용자들의 평점을 찾은 뒤 <strong>가중 평균</strong>하여, 현재 사용자의 누락 평점 대신에 사용된다
1. 사용자 $i$와 평가되지 않은 아이템 $j$를 특정한다
2. 유사도 척도 $s$를 사용하여 $j$아이템에 유사평점을 준 사용자 $K$를 찾는다 ($K$ 이웃의 수는 20~50 사이값을 사용한다)
3. $K$들의 평점을 가중평균하여, 사용자 $i$의 평점을 예측한다
4. 균질적인 평점을 비교하기 위하여, 평점분포를 정규화 한다
5. 예측평균이 1보다 작거나, 5보다 큰경우, 1과 5로 조정된다

In [15]:
def CF_userbased(u_vec, K, data, indxs = False):
    def FindKNeighbours(r, data, K):
        neighs, cnt = [], 0
        for u in range(len(data)):
            if data[u,r] > 0  and  cnt < K:
                neighs.append(data[u])
                cnt += 1 
            elif cnt == K: break
        return np.array(neighs)

    def CalcRating(u_vec, r, neighs):
        rating, den = 0., 0.
        for j in range(len(neighs)):
            rating += neighs[j][-1] * float(neighs[j][r] - neighs[j][neighs[j] > 0][:-1].mean())
            den += abs(neighs[j][-1])
        if den > 0:    rating = np.round(u_vec[u_vec>0].mean()+(rating/den),0)
        else:          rating = np.round(u_vec[u_vec>0].mean(),0)
        if rating > 5: return 5.
        elif rating<1: return 1.
        return rating 

    data = data.astype(float)
    nrows, ncols = len(data), len(data[0])
    data_sim = np.zeros((nrows,ncols+1))
    data_sim[:,:-1] = data

    # calc similarities:
    for u in range(nrows):
        if np.array_equal(data_sim[u, :-1], u_vec) == False: 
            data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec,'pearson')
        else:
            data_sim[u,ncols] = 0.

    #order by similarity:
    data_sim =data_sim[data_sim[:,ncols].argsort()][::-1]
    #find the K users for each item not rated:
    u_rec = np.zeros(len(u_vec))
    for r in xrange(ncols):
        if u_vec[r]==0:
            neighs = FindKNeighbours(r,data_sim,K)
            # calc the predicted ratin
            u_rec[r] = CalcRating(u_vec,r,neighs)

    #take out the rated movies
    if indxs:
        seenindxs = [indx for indx in xrange(len(u_vec)) if u_vec[indx]>0]
        u_rec[seenindxs] = -1
        recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        return recsvec    
    return u_rec